In [ ]:
import os
import numpy as np
from math import *
import warnings
warnings.filterwarnings("ignore")

#import datetime
import itertools
import uuid
from itertools import combinations
from random import randint

import numpy as np
# FOR LOADING DATA
from pandas import read_csv
from sklearn.utils import shuffle

# FOR utilities
from utils import *
# COMPUTE ALL PAIRS FROM SAMPLE SET
def all_pairs(sample_set):
    set_of_pairs = list(combinations(sample_set, 2))
    return set_of_pairs

def load_limited_dataset(filename,only_m_elements):
    dataset = read_csv(filename, header=None)
    rdataset = shuffle(dataset)
    sample_set = rdataset[:only_m_elements]
    data = sample_set.values
    X = data[:, :-1]
    y = data[:, -1]
    initial_size = data.shape[0]
    dimension = data.shape[1] - 1
    return data, X, y, dimension, initial_size
#from utils import *

# LOADING REAL DATASETS
folder_real_data = "realDatasets/"
data_filename = "data.csv"
filename12 = folder_real_data + "12-vote/" + data_filename

filename12_features = [
    "handicapped_infants",
    "water_project_cost_sharing",
    "adoption_of_the_budget_resolution",
    "physician_fee_freeze",
    "el_salvador_aid",
    "religious_groups_in_schools",
    "anti_satellite_test_ban",
    "aid_to_nicaraguan_contras",
    "mx_missile",
    "immigration",
    "synfuels_corporation_cutback",
    "education_spending",
    "superfund_right_to_sue",
    "crime",
    "duty_free_exports",
    "export_administration_act_south-africa",
]


# MAIN PROGRAM
filename_list = [filename12]
number_of_elements= 10
for dataset in filename_list:
    print("***************************************************************")
    _, full_dataset, _, dimension, initial_size = load_limited_dataset(dataset,435) # loading the full dataset for testing
    _, X, _, dimension, initial_size = load_limited_dataset(dataset,number_of_elements)
    print("1 Full dataset size:",len(full_dataset),"- Sample size:",initial_size,"- dimension:",dimension)
    set_of_pairs=all_pairs(X)
    print("2 Total number of pairs:",len(set_of_pairs),"- number of pairs of pairs:",len(set_of_pairs) * (len(set_of_pairs)-1))
    # MASK DEFINITIONS
    focus0=[1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0]
    focus1=[1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1]
    focus01= mask_intersection(focus0,focus1)
    # COMPUTING ANALOGIES WRT MASK
    list_of_analogy_on_focus0 =get_full_analogy_count(set_of_pairs,focus0)
    print("3 Total number of analogies found with focus0 =",focus0,":",len(list_of_analogy_on_focus0))
    
    list_of_analogy_on_focus1 =get_full_analogy_count(set_of_pairs,focus1)
    print("4 Total number of analogies found with focus1 =",focus1,":",len(list_of_analogy_on_focus1))
    
    list_of_analogy_on_focus01 =get_full_analogy_count(set_of_pairs,focus01)
    print("5 Total number of analogies found with mask intersection focus01 =",focus01,":",len(list_of_analogy_on_focus01))
    
    #print("Maximum number of combinations to be tested:", count_analogy0 * count_analogy1)
    print("*******************TESTING TRANSITIVITY***************************")
    candidate0,n0=check_transitivity(list_of_analogy_on_focus0,list_of_analogy_on_focus0, focus0)
    print("6 l0 with focus0, number of candidates:",candidate0,"number of successful transitivity:",n0)

    candidate1,n1=check_transitivity(list_of_analogy_on_focus1,list_of_analogy_on_focus1, focus1)
    print("7 l1 with focus1, number of candidates:",candidate1,"number of successful transitivity:",n1)

    candidate01_0,n01_0=check_transitivity(list_of_analogy_on_focus0,list_of_analogy_on_focus1, focus0)
    print("8 l0 and l1 with focus0, number of candidates:",candidate01_0,"number of successful transitivity:", n01_0)
    
    candidate01_1,n01_1=check_transitivity(list_of_analogy_on_focus0,list_of_analogy_on_focus1, focus1)
    print("9 l0 and l1 with with focus1, number of candidates:",candidate01_1,"number of successful transitivity:", n01_1)
    
    candidate01_01,n01_01=check_transitivity(list_of_analogy_on_focus0,list_of_analogy_on_focus1, focus01)
    print("10 l0 and l1 with with focus01, number of candidates:",candidate01_01,"number of successful transitivity:", n01_01)
    
    print("*******************TESTING UNICITY***************************")
    a,b,c,d = list_of_analogy_on_focus0[0] # Getting the first  solvable analogy wrt focus0
    #Solving the equation a:b::c:x wrt focus0 then focus1
    sol0=solve_vector_equation(a,b,c,focus0)
    sol1=solve_vector_equation(a,b,c,focus1)
    if (sol0==sol1):
        print("sol0=sol1")
    else:
        print("sol0 not equal sol1")
    matching_profile_focus0, matching_profile_focus1=0, 0
    for u in full_dataset:
        if matching_profile_on_mask(u,sol0,focus0):
            matching_profile_focus0+=1
        if matching_profile_on_mask(u,sol1,focus1):
            matching_profile_focus1+=1
    print("11 Solving a:b::c:d with focus0:",sol0)
    print("12 Solving a:b::c:d with focus1:",sol1)
    print("13 Number of matching_profile in full dataset with focus0:",matching_profile_focus0)
    print("14 Number of matching_profile in full dataset with focus1:",matching_profile_focus1)